In [125]:
import lasio
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

In [126]:
selected_columns = [
    "YYYY/MM/DD",
    "HH:MM:SS",
    "Hole Depth (feet)",
    "Bit Depth (feet)",
    "Rate Of Penetration (ft_per_hr)",
    "Weight on Bit (klbs)",
    "Hook Load (klbs)",
    "Rotary RPM (RPM)",
    "Standpipe Pressure (psi)",
    "Rotary Torque (kft_lb)",
    "Differential Pressure (psi)",
    "Flow (flow_percent)",
    "Total Pump Output (gal_per_min)",
    "Pason Gas (percent)",
    "On Bottom (unitless)",
    "Hole Diameter (in)",
    "Bit RPM (RPM)",
    "Bit Torque (kft_lb)",
    "EDR Instantaneous ROP (ft_per_hr)",
    "On Bottom ROP (ft_per_hr)",
    "Surface Stick Slip Index (percent)",
    "MWD Temp (Fahrenheit)",
    "Azimuth (degrees)",
    "Inclination (degrees)",
    "Temp IN (Degrees)",
    "Temp OUT (Degrees)",
    "AutoDriller WOB (klbs)",
    "AutoDriller Torque (kft_lb)",
    "Top Drive Rotary (RPM)",
    "Top Drive Torque (kft_lb)",
    "DAS BHA Stick Slip (unitless)",
    "DAS Bit Stick Slip (unitless)",
    "DAS MSE (ksi)",
    "DAS Downhole MSE (ksi)",
    "DAS ROP (ft_per_hr)",
    "DAS Recommended DIFFP (psi)",
    "DAS Recommended ROP (ft_per_hr)",
    "DAS Recommended RPM (RPM)",
    "DAS Recommended WOB (klbs)",
    "Depth of Cut (in)",
    "dexponent (log)",
    "Mechanical Specific Energy (ksi)",
    "TTS Mechanical Specific Energy (ksi)",
    "Relative MSE (unitless)"
]


In [ ]:
df1=pd.read_csv("27029986-3.csv", usecols=selected_columns)
df2=pd.read_csv("27029986-4.csv", usecols=selected_columns)
df3=pd.read_csv("27029986-5.csv", usecols=selected_columns)
df4=pd.read_csv("27029986-6.csv", usecols=selected_columns)

## Continiuty check over the dataframes

In [ ]:
for idx, data in enumerate([df1,df2,df3,df4]):
    print(f"Dataframe N{idx}")
    print("___________________")
    print("Size:", data.size)
    print("Shape:", data.shape)
    print("\n")

In [ ]:
df_list=[df1,df2,df3,df4]

for df in df_list:
    print("Record")
    print("Start Date: ", df["YYYY/MM/DD"][0])
    print("End Date: ",   df["YYYY/MM/DD"][df.shape[0]-1])
    print("Start Hole Depth: ", df["Hole Depth (feet)"][0])
    print("End Hole Depth: ", df["Hole Depth (feet)"][df.shape[0]-1])
    print("____________________________")


### CONCATING ALL RECORDS

In [ ]:
df=pd.concat([df1,df2,df3,df4], ignore_index=True)
df.head()

## Data Cleaning

- Missing data cleaning
- Duplication check
- Imputation

In [ ]:
df_mse=df[~df['DAS MSE (ksi)'].isnull()]

### Note 1 

- Currently it shows that there is no null value, but there are. -999.99 or similar value is a classic placeholder for missing or invalid sensor readings. First, I will first replace only -999.25 values with 0. Then, I will check new minimum values.

In [ ]:
df_mse.describe().T

### Data Cleaning Notes:Outlier and Placeholder Handling

- Replaced placeholder -999.25 with NaN across all numeric columns.

- Applied domain-specific sanity checks to filter out physically impossible or highly suspicious values:

  

| Column                  | Rule Applied                        | Reasoning                                                     |
| ----------------------- | ----------------------------------- | ------------------------------------------------------------- |
| `Temp IN (Degrees)`     | Removed values `< 32°F`             | Sub-freezing input temps are implausible                      |
| `Flow (flow_percent)`   | Kept only values in `[0%, 100%]`    | Flow % cannot be negative or exceed full capacity             |
| `Temp OUT (Degrees)`    | Kept only values in `[32°F, 400°F]` | Outside this range likely sensor drift or geothermal extremes |
| `Differential Pressure` | Removed values `< -500 psi`         | Large negative ∆P suggests reversed or broken signal          |
| `Bit Torque (kft_lb)`   | Removed negative values             | Torque can't be negative in this context                      |
| `Top Drive RPM`         | Removed negative values             | RPM directionality not relevant here; negatives = noise       |
| `Top Drive Torque`      | Removed negative values             | Same logic as Bit Torque                                      |
| `Depth of Cut (in)`     | Removed values `> 2 inches/cut`     | Extremely high values are data corruption or unit errors      |


In [ ]:
df_mse.head()

In [ ]:
def clean_drilling_data(df):
    df_clean = df.copy()

    # Replace all -999.25 placeholders with NaN
    df_clean = df_clean.replace(-999.25, np.nan)

    # Apply domain-specific filters
    df_clean.loc[df_clean['Temp IN (Degrees)'] < 32, 'Temp IN (Degrees)'] = np.nan
    df_clean.loc[(df_clean['Flow (flow_percent)'] < 0) | (df_clean['Flow (flow_percent)'] > 100), 'Flow (flow_percent)'] = np.nan
    df_clean.loc[(df_clean['Temp OUT (Degrees)'] < 32) | (df_clean['Temp OUT (Degrees)'] > 400), 'Temp OUT (Degrees)'] = np.nan
    df_clean.loc[df_clean['Bit Torque (kft_lb)'] < 0, 'Bit Torque (kft_lb)'] = np.nan
    df_clean.loc[df_clean['Top Drive Rotary (RPM)'] < 0, 'Top Drive Rotary (RPM)'] = np.nan
    df_clean.loc[df_clean['Top Drive Torque (kft_lb)'] < 0, 'Top Drive Torque (kft_lb)'] = np.nan
    df_clean.loc[df_clean['Depth of Cut (in)'] > 2, 'Depth of Cut (in)'] = np.nan
    df_clean.loc[(df_clean['Rate Of Penetration (ft_per_hr)'] < 0) | (df_clean['Rate Of Penetration (ft_per_hr)'] > 500), 'Temp OUT (Degrees)'] = np.nan
    
    df_clean.loc[df_clean['Rate Of Penetration (ft_per_hr)'] > 500, 'Depth of Cut (in)'] = np.nan
    

    return df_clean
df_mse_clean = clean_drilling_data(df_mse)
df_mse_clean.drop(columns=['Pason Gas (percent)','DAS Recommended DIFFP (psi)'], inplace=True)

In [ ]:
df_mse_clean['Rate Of Penetration (ft_per_hr)'].max()

In [ ]:
df_mse_clean.describe().T

### Additional Cleaning Notes: Extreme Outlier Removal for MSE & ROP

- Detected physically impossible max values in key drilling metrics (e.g., ROP > 11,000 ft/hr, MSE > 40 billion ksi).

- These are likely due to sensor glitches, divide-by-zero errors, or corrupted logging data.

- Applied hard caps based on realistic operational ranges to safely remove outliers:

In [ ]:
import pandas as pd
import numpy as np

def clean_extreme_mse_rop(df):
    df_filtered = df.copy()

    # Cap ROPs
    rop_columns = [
        'Rate Of Penetration (ft_per_hr)',
        'EDR Instantaneous ROP (ft_per_hr)',
        'On Bottom ROP (ft_per_hr)'
    ]
    for col in rop_columns:
        if col in df_filtered.columns:
            df_filtered.loc[df_filtered[col] > 300, col] = np.nan

    # Cap MSE values
    mse_columns = [
        'DAS MSE (ksi)',
        'Mechanical Specific Energy (ksi)',
        'TTS Mechanical Specific Energy (ksi)'
    ]
    for col in mse_columns:
        if col in df_filtered.columns:
            df_filtered.loc[df_filtered[col] > 1000, col] = np.nan

    # Cap Relative MSE
    if 'Relative MSE (unitless)' in df_filtered.columns:
        df_filtered.loc[df_filtered['Relative MSE (unitless)'] > 20, 'Relative MSE (unitless)'] = np.nan

    return df_filtered


In [ ]:
df_mse_clean=clean_extreme_mse_rop(df_mse_clean)

In [ ]:
df_mse_clean.describe().T

In [ ]:
def plot_on_bottom_overlay(df):
    sns.set(style="whitegrid")
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

    # Plot ROP vs Bit Depth
    sns.scatterplot(ax=axes[0],
                    data=df,
                    x="Rate Of Penetration (ft_per_hr)",
                    y="Bit Depth (feet)",
                    hue="On Bottom (unitless)",
                    palette={0: "red", 1: "green"},
                    s=5,
                    alpha=0.5)
    axes[0].invert_yaxis()
    axes[0].set_title("ROP vs Bit Depth (colored by On Bottom)")
    axes[0].set_xlabel("ROP (ft/hr)")
    axes[0].set_ylabel("Bit Depth (feet)")

    # Plot Hook Load vs Bit Depth
    sns.scatterplot(ax=axes[1],
                    data=df,
                    x="Hook Load (klbs)",
                    y="Bit Depth (feet)",
                    hue="On Bottom (unitless)",
                    palette={0: "red", 1: "green"},
                    s=5,
                    alpha=0.5)
    axes[1].invert_yaxis()
    axes[1].set_title("Hook Load vs Bit Depth (colored by On Bottom)")
    axes[1].set_xlabel("Hook Load (klbs)")
    axes[1].set_ylabel("Bit Depth (feet)")

    plt.tight_layout()
    plt.show()


In [ ]:
plot_on_bottom_overlay(df_mse_clean)

In [ ]:
df_mse_clean['On Bottom (unitless)'].value_counts(normalize=True)

In [ ]:
df_on_bottom=df_mse_clean[df_mse_clean['On Bottom (unitless)']==1]

In [ ]:
plot_on_bottom_overlay(df_on_bottom)

In [ ]:
df_on_bottom["Relative MSE (unitless)"].value_counts()

In [ ]:
df_eff = df_on_bottom[(df_on_bottom["Relative MSE (unitless)"] > 0) & (df_on_bottom["Relative MSE (unitless)"] < 1.2)]
df_ineff = df_on_bottom[df_on_bottom["Relative MSE (unitless)"] > 2.0]

In [ ]:
df_eff['Rotary RPM (RPM)'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_eff, x="Weight on Bit (klbs)", y="Rotary RPM (RPM)", fill=True, cmap="Greens", alpha=0.6, label="Efficient")
sns.kdeplot(data=df_ineff, x="Weight on Bit (klbs)", y="Rotary RPM (RPM)", fill=True, cmap="Reds", alpha=0.3, label="Inefficient")
plt.legend()
plt.xlim(20, 60)
plt.ylim(0, 20)
plt.title("KDE of WOB vs RPM: Efficient vs Inefficient Drilling")
plt.xlabel("WOB (klbs)")
plt.ylabel("RPM")
plt.show()


In [ ]:
df_eff["Weight on Bit (klbs)"].value_counts().head(10)

In [ ]:
df_eff[df_eff["Weight on Bit (klbs)"] < 1].shape

In [124]:
df_eff = df_eff[df_eff["Weight on Bit (klbs)"] > 2]